In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [2]:
data_path_file='DelayedFlights.csv'
df =pd.read_csv(data_path_file)

Centraremos el analisis en las caracteristicas de interés: features y lo guardaremos en el dataframe X

In [3]:
features=[ 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'ActualElapsedTime',
          'CRSElapsedTime', 'AirTime', 'ArrDelay','DepDelay', 'Distance', 'Cancelled', 'Diverted', 'UniqueCarrier']

In [4]:
X= df[features]

In [5]:
X.shape

(1936758, 14)

Dado:

* la variablidad en los retrasos por compañias 
* el gran numero de datos disponibles
* Incluir la variable categorica 'UniqueCarrier' con 20 valores diferentes con un One-Hot Encoding seria ineficiente.

Escogeremos una compañia para realizar nuestro modelo.

Supongamos que la segunda compañia en volumen de vuelos American Airlines 'AA' como parte de una estrategia de mejora quiere obtener un modelo de los retrasos que tiene.

In [6]:
X = X[ X['UniqueCarrier'] == 'AA' ]

In [7]:
X.shape

(191865, 14)

Supongamos que **queremos entrenar un modelo que sea capaz de estimar la caracteristica 'ArrDelay' una vez el vuelo ya ha salido pero obviamente aun no ha llegado**. 

Para ello 'Arrdelay' no ha de tener faltantes. tratandose de un numero relativamente bajo descartaremos las filas con 'ArrDelay' faltantes para toda la matriz de caracteristicas X

In [8]:
nodelay = ~(X['ArrDelay'].isnull())
X= X[nodelay]

In [9]:
X.shape

(190910, 14)

Guardamos en la variable y la caracteristica 'ArrDelay' y la eliminaremos de la matriz de caracteristicas X

In [10]:
y= X.ArrDelay
X.drop( columns='ArrDelay', inplace= True )

In [11]:
X.shape

(190910, 13)

Por tanto las caracteristicas de la matrix X seran las contenidas en la lista feat_X. Por ser nulas Cancelled y Diverted se eliminan tambien. **Como el vuelo no ha llegado en el momento de estimar su retraso, no dispondremos de 'ArrTime', 'ActualElapsedTime' ni 'Airtime'**

In [12]:
feat_X=[ 'DayOfWeek', 'DepTime', 'CRSDepTime', 'CRSArrTime',
          'CRSElapsedTime','DepDelay', 'Distance']

In [13]:
X= X[feat_X]

In [14]:
X.shape

(190910, 7)

Para los modelos Lineales y RandomForest no es necesario standardizar los datos. No obstante **para el modelo MLPRegressor si es necesario para mejorar la convergencia.** Asi pues **standardizaremos los datos de la matriz X**

Como se standardiza en base a los datos de train como paso previo separamos con train_test_split()

In [15]:
# Dividimos los datos (X,y) en training y test (X_train, X_test, y_train, y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(152728, 7) (38182, 7) (152728,) (38182,)


Standardizamos ambos dataframes: X_train, X_test

In [16]:
scaler = preprocessing.StandardScaler()

In [17]:
# Fit to TRAIN data. Then transform it.
M_train = scaler.fit_transform(X_train)
#Volvemos a meter la matriz en dataframe
X_train = pd.DataFrame(M_train, columns= feat_X )

# Transform TEST data 
M_test = scaler.transform(X_test)
#Volvemos a meter la matriz en dataframe
X_test= pd.DataFrame(M_test, columns= feat_X )

In [18]:
print(X_train.shape, X_test.shape)

(152728, 7) (38182, 7)


--------------------------
## Ejercicio 1


Crea almenos 3 modelos de Regresion diferentes para intentar predecir el retraso de los vuelos 'ArrDelay'

-------------------------


Usaremos los siguientes modelos:

* Modelo Lineal
* Random Forest
* MLP Regressor

Generaremos instancias de los modelos, los entrenaremos y realizaremos las predicciones para todos ellos con los parametros por defecto.

In [19]:
# Modelo Lineal
linear_1 = LinearRegression()

#Entrenamos el modelo lineal
linear_1.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_lin1= linear_1.predict(X_test)

In [20]:
# Modelo Random Forest
forest_1 = RandomForestRegressor()

#Entrenamos el Forest Regressor
forest_1.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_for1 = forest_1.predict(X_test)

In [21]:
# Modelo MLPRegressor
mlp_1 = MLPRegressor()

#Entrenamos el MLP Regressor
mlp_1.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_mlp1 = mlp_1.predict(X_test)

c:\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


--------------------------
## Ejercicio 2

Compara los modelos segun el MSE y el R2.

--------------------------

Evaluaremos la calidad del modelo con los datos de test

In [22]:
r2_lin1 = linear_1.score(X_test, y_test)
mse_lin1 = mean_squared_error(y_test, y_pred_lin1)

r2_for1 = forest_1.score(X_test, y_test)
mse_for1 = mean_squared_error(y_test, y_pred_for1)

r2_mlp1= mlp_1.score(X_test, y_test)
mse_mlp1 = mean_squared_error(y_test, y_pred_mlp1)

In [23]:
print('Linear Regression | R^2: ', round(r2_lin1,6), ' | ', 'MSE: ', round(mse_lin1,6) )
print('Random Forest     | R^2: ', round(r2_for1,6), ' | ', 'MSE: ', round(mse_for1,6) )
print('MLPRegressor      | R^2: ', round(r2_mlp1,6), ' | ', 'MSE: ', round(mse_mlp1,6) )

Linear Regression | R^2:  0.906291  |  MSE:  341.136325
Random Forest     | R^2:  0.898135  |  MSE:  370.827487
MLPRegressor      | R^2:  0.908065  |  MSE:  334.676596


In [24]:
linear_1.coef_

array([-0.61975558,  0.70175638, -1.34389083,  0.17025827,  0.85015817,
       58.50323224, -1.71995203])

El orden de los modelos de mejor a peor serian el MLPRegressor, Regresion Lineal y por ultimo Random Forest. Orden creciente de R^2 y decreciente de MSE.

---------------------------

## Ejercicio 3


Utiliza los diferentes parametros que admiten

----------------------------------------------

Para el modelo lineal implementaremos la variante Ridge que incluye en la funcion a minimizar la suma de los coeficientes al cuadrado.

Ridge Linear Regression:

* alpha: factor sobre la suma de coeficientes del modelo al cuadrado. alpha=0 seria el modelo lineal standard.

Random Forest:
* n_estimators: numero de arboles del modelo
* max_depth: Maximos niveles de division del modelo
* min_samples_split: Numero minimo de muestras que ha de tener el nodo para poder hacer split. 
* min_samples_leaf: Solo se hará split del nodo si deja como minimo min_samples_leaf en ambas ramas.
* criterion: El criterio de optimizacion. Por defecto es el squared error.
* n_jobs: Divide la computacion del modelo en los nucleos del procesador del ordenador local, 4 en mi caso.

MLP Regressor:
* hidden_layer_sizes: Tamaño de las capas ocultas. Se da en tupla. La red ademas tendra una capa incial y la final de salida.
* activation: Funcion de activacion. Por defecto la 'relu'
* solver: El solver tiene 3 opciones la default es 'adam'
* learning_rate: fija el tipo de actualizacion de los pesos, por default='constant'.
* learning_rate_init: El valor inicial del learning_rate es 0.001
* tol: Itera hasta la convergencia marcada por este parametro, por defecto 1E-4.  
* max_iter: Si no llega a la convergencia para en la iteracion marcada por este parametro

In [25]:
linear_2 = Ridge(alpha=1, random_state=33)

#Entrenamos el modelo lineal
linear_2.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_lin2 = linear_2.predict(X_test)

In [26]:
# Modelo Random Forest
forest_2 = RandomForestRegressor(n_estimators = 200, 
                               max_depth=10, min_samples_split= 50, min_samples_leaf=20,
                               random_state = 33, criterion= 'squared_error', n_jobs=4)

#Entrenamos el Forest Regressor
forest_2.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_for2 = forest_2.predict(X_test)

In [27]:
# Modelo MLPRegressor
mlp_2 = MLPRegressor(hidden_layer_sizes= (80,40,20), activation='relu', solver='adam',
                      learning_rate_init= 0.05, learning_rate='adaptive', max_iter=100,
                      random_state=33)

#Entrenamos el MLP Regressor
mlp_2.fit(X_train, y_train)

#Predecimos a partir de los datos de test
y_pred_mlp2 = mlp_2.predict(X_test)

In [28]:
r2_lin2 = linear_2.score(X_test, y_test)
mse_lin2 = mean_squared_error(y_test, y_pred_lin2)

r2_for2 = forest_2.score(X_test, y_test)
mse_for2 = mean_squared_error(y_test, y_pred_for2)

r2_mlp2= mlp_2.score(X_test, y_test)
mse_mlp2 = mean_squared_error(y_test, y_pred_mlp2)

In [29]:
print('Linear Regression | R^2: ', round(r2_lin1,6), ' | ', 'MSE: ', round(mse_lin1,6) )
print('Ridge Linear Reg  | R^2: ', round(r2_lin2,6), ' | ', 'MSE: ', round(mse_lin2,6) )
print('Random Forest 1   | R^2: ', round(r2_for1,6), ' | ', 'MSE: ', round(mse_for1,6) )
print('Random Forest 2   | R^2: ', round(r2_for2,6), ' | ', 'MSE: ', round(mse_for2,6) )
print('MLPRegressor  1   | R^2: ', round(r2_mlp1,6), ' | ', 'MSE: ', round(mse_mlp1,6) )
print('MLPRegressor  2   | R^2: ', round(r2_mlp2,6), ' | ', 'MSE: ', round(mse_mlp2,6) )

Linear Regression | R^2:  0.906291  |  MSE:  341.136325
Ridge Linear Reg  | R^2:  0.906291  |  MSE:  341.136301
Random Forest 1   | R^2:  0.898135  |  MSE:  370.827487
Random Forest 2   | R^2:  0.906687  |  MSE:  339.693849
MLPRegressor  1   | R^2:  0.908065  |  MSE:  334.676596
MLPRegressor  2   | R^2:  0.906442  |  MSE:  340.585975


--------------------

## Ejercicio 4

Compara el rendimiento entre usar o no usar el train/test

----------------------

In [30]:
#Standardizamos todo el database
scaler2 = preprocessing.StandardScaler()
# Fit to ALL data. Then transform it.
M = scaler2.fit_transform(X)
#Volvemos a meter la matriz en el dataframe XX
XX = pd.DataFrame(M, columns= feat_X )

In [31]:
XX.shape

(190910, 7)

In [32]:
# Modelo Lineal
model1 = LinearRegression(n_jobs=4)
model1.fit(XX, y)
y_pred_1= model1.predict(XX)

# Modelo Random Forest
model2 = RandomForestRegressor(n_estimators = 200, 
                               max_depth=10, min_samples_split= 50, min_samples_leaf=20,
                               random_state = 33, criterion= 'squared_error', n_jobs=4)
model2.fit(XX, y)
y_pred_2= model2.predict(XX)

# Modelo MLPRegressor
model3 = MLPRegressor(hidden_layer_sizes= (80,40,20), activation='relu', solver='adam',
                      learning_rate_init= 0.05, learning_rate='adaptive', max_iter=100,
                      random_state=33)
model3.fit(XX, y)
y_pred_3= model3.predict(XX)

Evaluamos las predicciones

In [33]:
r2_m1= model1.score(XX, y)
mse1 = mean_squared_error(y, y_pred_1)
r2_m2= model2.score(XX, y)
mse2 = mean_squared_error(y, y_pred_2)
r2_m3= model3.score(XX, y)
mse3 = mean_squared_error(y, y_pred_3)

print('Linear Reg. All Data | R^2: ', round(r2_m1,6), '  | ', 'MSE:', round(mse1,6) )
print('Ridge Linear Test D  | R^2: ', round(r2_lin2,6), ' | ', 'MSE: ', round(mse_lin2,6) )
print('Forest Reg. All Data | R^2: ', round(r2_m2,6), ' | ', 'MSE:', round(mse2,6) )
print('Random For2  Test D  | R^2: ', round(r2_for2,6), ' | ', 'MSE: ', round(mse_for2,6) )
print('MLP Reg.  All Data   | R^2: ', round(r2_m3,6), ' | ', 'MSE:', round(mse3,6) )
print('MLPRegressor  2      | R^2: ', round(r2_mlp2,6), ' | ', 'MSE: ', round(mse_mlp2,6) )

Linear Reg. All Data | R^2:  0.90566   |  MSE: 354.044691
Ridge Linear Test D  | R^2:  0.906291  |  MSE:  341.136301
Forest Reg. All Data | R^2:  0.909153  |  MSE: 340.937218
Random For2  Test D  | R^2:  0.906687  |  MSE:  339.693849
MLP Reg.  All Data   | R^2:  0.906488  |  MSE: 350.937601
MLPRegressor  2      | R^2:  0.906442  |  MSE:  340.585975


En general hacer la validacion con los mismos datos con los que se entrena el modelo dara R^2 mayor y MSE menor. Esto indicaria que las predicciones son mejores. No obstante es mejor hacer un test del modelo con datos que no hayan intervenido en la construccion del modelo. 

Asi se puede valorar la prediccion de datos que el modelo no ha usado. El sobre-entreno o 'Over Fitting' puede llevar a un modelo que prediga con precision los datos con los que fue construido pero le resulte dificil realizar una buena prediccion de datos que aun no ha visto.

-----------------------

## Ejercicio 5

Realiza algun proceso de Ingenieria de Variables para mejorar la prediccion.

-----------------------

Añadiremos la mejor variable predictora 'DepDelay' al cuadrado y al cubo en la matriz de caracteristicas. 

In [34]:
X_fe = X

In [35]:
feat_X_fe = feat_X
feat_X_fe.append('DD2')
feat_X_fe.append('DD3')
feat_X_fe

['DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'CRSArrTime',
 'CRSElapsedTime',
 'DepDelay',
 'Distance',
 'DD2',
 'DD3']

In [36]:
X_fe['DD2']=X_fe['DepDelay']**2

In [37]:
X_fe['DD3']=X_fe['DepDelay']**3

In [38]:
X_fe.head()

,DayOfWeek,DepTime,CRSDepTime,CRSArrTime,CRSElapsedTime,DepDelay,Distance,DD2,DD3
145160,3,906.0,900,1225,385.0,6.0,2475,36.0,216.0
145161,4,915.0,900,1225,385.0,15.0,2475,225.0,3375.0
145162,4,918.0,900,1225,385.0,18.0,2475,324.0,5832.0
145163,5,922.0,900,1225,385.0,22.0,2475,484.0,10648.0
145164,5,946.0,930,1750,320.0,16.0,2475,256.0,4096.0


In [39]:
# Dividimos los datos (X,y) en training y test (X_train, X_test, y_train, y_test)

X_fe_train, X_fe_test, y_train, y_test = train_test_split(X_fe, y, train_size=0.8, test_size=0.2, random_state=33)
print(X_fe_train.shape, X_fe_test.shape, y_train.shape, y_test.shape)

(152728, 9) (38182, 9) (152728,) (38182,)


In [40]:
scaler3 = preprocessing.StandardScaler()

In [41]:
# Fit to TRAIN data. Then transform it.
M_fe_train = scaler3.fit_transform(X_fe_train)
#Volvemos a meter la matriz en dataframe
X_fe_train = pd.DataFrame(M_fe_train, columns= feat_X_fe )

# Transform TEST data 
M_fe_test = scaler3.transform(X_fe_test)
#Volvemos a meter la matriz en dataframe
X_fe_test= pd.DataFrame(M_fe_test, columns= feat_X_fe )

In [42]:
# Modelo Lineal
model1b = LinearRegression(n_jobs=4)

#Entrenamos el modelo lineal
model1b.fit(X_fe_train, y_train)

y_pred_1b= model1b.predict(X_fe_test)

In [43]:
r2_m1b= model1b.score(X_fe_test, y_test)
mse1b = mean_squared_error(y_test, y_pred_1b)

print('Linear Regression X_fe Validation Results: R^2:', round(r2_m1b,6), 'MSE:', round(mse1b,6) )

Linear Regression X_fe Validation Results: R^2: 0.906407 MSE: 340.715051


Observamos como la introduccion de 'DepDelay' al cuadrado y al cubo ha mejorado levemente la prediccion del modelo. El modelo lineal sin estas variables habia dado R^2:0.906291 y MSE:341.1363. Observamos un valor de R^2 mayor y menor MSE para las predicciones realizadas sobre el conjunto de datos de test.



----------------------------

## Ejercicio 6

No utilices la variable 'DepDelay' en las prediciones

----------------------------

Definamos la lista de variables a usar X_train_red y X_test_red 

In [44]:
X_train_red = X_train.drop( columns='DepDelay', inplace= False )
X_test_red = X_test.drop(columns='DepDelay', inplace=False )

In [45]:
# Modelo Lineal
model1c = LinearRegression(n_jobs=4)

# Modelo Random Forest
model2c = RandomForestRegressor(n_estimators = 200, 
                               max_depth=10, min_samples_split= 50, min_samples_leaf=20,
                               random_state = 33, criterion= 'squared_error', n_jobs=4)

# Modelo MLPRegressor
model3c = MLPRegressor(hidden_layer_sizes= (20,10,), activation='relu', solver='adam',
                      learning_rate_init= 0.05, learning_rate='adaptive', max_iter=100,
                      random_state=33)

In [46]:
#Entrenamos el modelo lineal
model1c.fit(X_train_red, y_train)


LinearRegression(n_jobs=4)

In [47]:
y_pred_1c= model1c.predict(X_test_red)
r2_m1c = model1c.score(X_test_red, y_test)
mse1c = mean_squared_error(y_test, y_pred_1c)

print('Linear Regression noDepDelay Validation Results: R^2:', round(r2_m1c,6), 'MSE:', round(mse1c,6) )

Linear Regression noDepDelay Validation Results: R^2: 0.024437 MSE: 3551.414223


Claramente obtenemos peor resultado en las predicciones. Menor R^2 y mayor MSE